In [246]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [247]:
data = pd.read_csv("C:/Users/Meyke/Documents/Projecten/Vendee/0_85af9c3ad0944b8aac9dfac7666b8674_1.csv")

In [248]:
del data['Unnamed: 0']

In [249]:
data.tail(10)

,Rank,time,Distance Travelled,boat,latitude,longitude,heading,speed,EventProcessedUtcTime,PartitionId,EventEnqueuedUtcTime,Distance To Front Runner
3320,1,1630608658,165.075878,9,37.594073,-10.750392,228.470856,10.111671,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,0.000000
3321,2,1630608658,149.838910,7,37.689256,-10.625974,212.312824,15.266012,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,15.237231
3322,3,1630608658,146.235339,8,37.711334,-10.595991,227.913756,11.733489,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,18.841403
3323,4,1630608658,139.721101,0,37.747979,-10.537711,238.834699,14.643653,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,25.370776
3324,5,1630608658,121.048011,3,37.843140,-10.357326,212.339294,2.151606,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,44.311954
3325,6,1630608658,89.835278,4,38.053836,-10.121600,240.672691,7.652952,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,75.281285
3326,7,1630608658,59.108595,6,38.270832,-9.899322,224.505695,13.911123,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,106.024156
3327,8,1630608658,55.739380,1,38.256729,-9.827544,226.661457,9.378078,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,109.492635
3328,9,1630608658,55.303027,2,38.274652,-9.844028,215.303118,5.070161,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,109.785378
3329,10,1630608658,44.310144,5,38.330311,-9.736073,213.938856,17.527448,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,120.905551


In [250]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3330 entries, 0 to 3329
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Rank                      3330 non-null   int64  
 1   time                      3330 non-null   int64  
 2   Distance Travelled        3330 non-null   float64
 3   boat                      3330 non-null   int64  
 4   latitude                  3330 non-null   float64
 5   longitude                 3330 non-null   float64
 6   heading                   3330 non-null   float64
 7   speed                     3330 non-null   float64
 8   EventProcessedUtcTime     3330 non-null   object 
 9   PartitionId               3330 non-null   int64  
 10  EventEnqueuedUtcTime      3330 non-null   object 
 11  Distance To Front Runner  3330 non-null   float64
dtypes: float64(6), int64(4), object(2)
memory usage: 312.3+ KB


In [251]:
DistanceFirstLM = {}

# Calculate total distance travelled
for time in data.time.unique():
    DistanceFirstLM[time] = {}
    for boat in data.boat.unique():
        #radius of the Earth
        R = 6373.0
        
        dataN = data[data['time'] == time]

        #coordinates
        lat1 = math.radians(15)
        lon1 = math.radians(-20)
        lat2 = math.radians(dataN[dataN['boat'] == boat].iloc[-1,4])
        lon2 = math.radians(dataN[dataN['boat'] == boat].iloc[-1,5])

        #change in coordinates
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        #change in coordinates
        a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2

        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = R * c

        DistanceFirstLM[time][boat] = distance

In [252]:
TotalTravelled2 = pd.DataFrame.from_dict({(i,j): DistanceFirstLM[i][j] 
                           for i in DistanceFirstLM.keys() 
                           for j in DistanceFirstLM[i].keys()},
                       orient='index')

In [253]:
TotalTravelled2.tail(10)

,0
"(1630608658, 4)",2741.995520
"(1630608658, 1)",2772.983098
"(1630608658, 5)",2783.703719
"(1630608658, 6)",2771.889882
"(1630608658, 3)",2712.302968
"(1630608658, 8)",2690.641608
"(1630608658, 0)",2696.380618
"(1630608658, 7)",2687.359627
"(1630608658, 9)",2673.384062
"(1630608658, 2)",2774.224431


In [254]:
TotalTravelled2 = TotalTravelled2.reset_index()

In [255]:
TotalTravelled2 = TotalTravelled2.rename(columns={'index': 'Time-Boat', 0:'Distance 1st landmark'})

In [256]:
TotalTravelled2['time'] = TotalTravelled2.apply(lambda row: row['Time-Boat'][0], axis=1)

In [257]:
TotalTravelled2['boat'] = TotalTravelled2.apply(lambda row: row['Time-Boat'][1], axis=1)

In [258]:
del TotalTravelled2['Time-Boat']

In [259]:
RankDistanceTravelled = {}

# Calculate total distance travelled
for time in data.time.unique():
    RankDistanceTravelled[time] = {}
    TotalTravelled2N = TotalTravelled2[TotalTravelled2['time'] == time]
    listDistances = TotalTravelled2N['Distance 1st landmark'].to_list()
    listDistances.sort(reverse=False)
    for distance in listDistances:
        RankDistanceTravelled[time][distance] = listDistances.index(distance) + 1

In [260]:
TotalTravelled3 = pd.DataFrame.from_dict({(i,j): RankDistanceTravelled[i][j] 
                           for i in RankDistanceTravelled.keys() 
                           for j in RankDistanceTravelled[i].keys()},
                       orient='index')

In [261]:
TotalTravelled3 = TotalTravelled3.reset_index()
TotalTravelled3 = TotalTravelled3.rename(columns={'index': 'Time-Distance', 0:'Rank 1LM'})

TotalTravelled3['time'] = TotalTravelled3.apply(lambda row: row['Time-Distance'][0], axis=1)
TotalTravelled3['Distance 1st landmark'] = TotalTravelled3.apply(lambda row: row['Time-Distance'][1], axis=1)

In [262]:
del TotalTravelled3['Time-Distance']

In [263]:
TotalTravelled3.head(10)

,Rank 1LM,time,Distance 1st landmark
0,1,1630568818,2824.435159
1,2,1630568818,2824.723581
2,3,1630568818,2824.919089
3,4,1630568818,2825.432192
4,5,1630568818,2825.683277
5,6,1630568818,2825.912268
6,7,1630568818,2826.020685
7,8,1630568818,2826.185478
8,9,1630568818,2826.604894
9,10,1630568818,2826.841913


In [264]:
TotalTravelled = TotalTravelled3.merge(TotalTravelled2, on=['time','Distance 1st landmark'])

In [265]:
TotalTravelled = TotalTravelled.merge(data, on=['time','boat'])

In [266]:
TotalTravelled

,Rank 1LM,time,Distance 1st landmark,boat,Rank,Distance Travelled,latitude,longitude,heading,speed,EventProcessedUtcTime,PartitionId,EventEnqueuedUtcTime,Distance To Front Runner
0,1,1630568818,2824.435159,4,1,1.427177,38.614716,-9.403606,230.835196,8.755974,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,0.000000
1,2,1630568818,2824.723581,1,2,1.124782,38.614134,-9.393962,243.595338,5.663308,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,0.840667
2,3,1630568818,2824.919089,5,3,1.058820,38.614806,-9.390425,223.677814,4.822775,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,1.145628
3,4,1630568818,2825.432192,9,9,0.405287,38.621923,-9.396180,234.620743,13.367485,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,1.029174
4,5,1630568818,2825.683277,7,8,0.658712,38.620867,-9.386242,205.598402,12.608633,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,1.656916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3325,6,1630608658,2741.995520,4,6,89.835278,38.053836,-10.121600,240.672691,7.652952,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,75.281285
3326,7,1630608658,2771.889882,6,7,59.108595,38.270832,-9.899322,224.505695,13.911123,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,106.024156
3327,8,1630608658,2772.983098,1,8,55.739380,38.256729,-9.827544,226.661457,9.378078,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,109.492635
3328,9,1630608658,2774.224431,2,9,55.303027,38.274652,-9.844028,215.303118,5.070161,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,109.785378


In [267]:
DistanceFrontRunner = {}

# Compute distance between boats and front-runner
for time in data.time.unique():
    DistanceFrontRunner[time] = {}
    TotalTravelledXN = TotalTravelled[TotalTravelled['time'] == time]
    
    # Get FrontRunner Boat
    FrontRunner = TotalTravelledXN[TotalTravelledXN['Rank']==1].iloc[0,3]
    # Get Long & Lat FrontRunner
    FrLong = TotalTravelledXN[TotalTravelledXN['boat'] == FrontRunner].iloc[-1,7]
    FrLat = TotalTravelledXN[TotalTravelledXN['boat'] == FrontRunner].iloc[-1,6]
    
    DistanceFrontRunner[time] = {}
    
    # Calculate total distance travelled

    for boat in data.boat.unique():
        #radius of the Earth
        R = 6373.0

        #coordinates
        lat1 = math.radians(FrLat)
        lon1 = math.radians(FrLong)
        lat2 = math.radians(TotalTravelledXN[TotalTravelledXN['boat'] == boat].iloc[-1,6])
        lon2 = math.radians(TotalTravelledXN[TotalTravelledXN['boat'] == boat].iloc[-1,7])

        #change in coordinates
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        #change in coordinates
        a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2

        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = R * c

        DistanceFrontRunner[time][boat] = distance

In [268]:
TotalTravelled4 = pd.DataFrame.from_dict({(i,j): DistanceFrontRunner[i][j] 
                           for i in DistanceFrontRunner.keys() 
                           for j in DistanceFrontRunner[i].keys()},
                       orient='index')

In [269]:
TotalTravelled4

,0
"(1630568818, 4)",0.000000
"(1630568818, 1)",0.840667
"(1630568818, 5)",1.145628
"(1630568818, 6)",2.466575
"(1630568818, 3)",1.795394
...,...
"(1630608658, 8)",18.841403
"(1630608658, 0)",25.370776
"(1630608658, 7)",15.237231
"(1630608658, 9)",0.000000


In [270]:
TotalTravelled4 = TotalTravelled4.reset_index()
TotalTravelled4 = TotalTravelled4.rename(columns={'index': 'Time-Boat', 0:'Distance To Front Runner 1LM'})

TotalTravelled4['time'] = TotalTravelled4.apply(lambda row: row['Time-Boat'][0], axis=1)
TotalTravelled4['boat'] = TotalTravelled4.apply(lambda row: row['Time-Boat'][1], axis=1)

In [271]:
del TotalTravelled4['Time-Boat']

In [272]:
TotalTravelled = TotalTravelled.merge(TotalTravelled4, on=['time','boat'])

In [273]:
TotalTravelled

,Rank 1LM,time,Distance 1st landmark,boat,Rank,Distance Travelled,latitude,longitude,heading,speed,EventProcessedUtcTime,PartitionId,EventEnqueuedUtcTime,Distance To Front Runner,Distance To Front Runner 1LM
0,1,1630568818,2824.435159,4,1,1.427177,38.614716,-9.403606,230.835196,8.755974,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,0.000000,0.000000
1,2,1630568818,2824.723581,1,2,1.124782,38.614134,-9.393962,243.595338,5.663308,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,0.840667,0.840667
2,3,1630568818,2824.919089,5,3,1.058820,38.614806,-9.390425,223.677814,4.822775,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,1.145628,1.145628
3,4,1630568818,2825.432192,9,9,0.405287,38.621923,-9.396180,234.620743,13.367485,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,1.029174,1.029174
4,5,1630568818,2825.683277,7,8,0.658712,38.620867,-9.386242,205.598402,12.608633,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,1.656916,1.656916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3325,6,1630608658,2741.995520,4,6,89.835278,38.053836,-10.121600,240.672691,7.652952,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,75.281285,75.281285
3326,7,1630608658,2771.889882,6,7,59.108595,38.270832,-9.899322,224.505695,13.911123,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,106.024156,106.024156
3327,8,1630608658,2772.983098,1,8,55.739380,38.256729,-9.827544,226.661457,9.378078,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,109.492635,109.492635
3328,9,1630608658,2774.224431,2,9,55.303027,38.274652,-9.844028,215.303118,5.070161,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,109.785378,109.785378


In [274]:
DistanceFrontRunner2 = {}

# Compute distance between boats and front-runner
for time in data.time.unique():
    DistanceFrontRunner2[time] = {}
    TotalTravelledXN = TotalTravelled[TotalTravelled['time'] == time]
    
    # Get FrontRunner Boat
    FrontRunner = TotalTravelledXN[TotalTravelledXN['Rank']==1].iloc[0,3]
    # Get distance to landmark Front Runner
    DLM_fr = TotalTravelledXN[TotalTravelledXN['boat'] == FrontRunner].iloc[-1,2]
    
    # Difference with front runner

    for boat in data.boat.unique():
        
        DLM_boat = TotalTravelledXN[TotalTravelledXN['boat'] == boat].iloc[-1,2]
    
        differenceInDistance = round(DLM_boat - DLM_fr,4)

        DistanceFrontRunner2[time][boat] = differenceInDistance

In [275]:
TotalTravelled5 = pd.DataFrame.from_dict({(i,j): DistanceFrontRunner2[i][j] 
                           for i in DistanceFrontRunner2.keys() 
                           for j in DistanceFrontRunner2[i].keys()},
                       orient='index')

In [276]:
TotalTravelled5 = TotalTravelled5.reset_index()
TotalTravelled5 = TotalTravelled5.rename(columns={'index': 'Time-Boat', 0:'Difference Compared To Front Runner'})

TotalTravelled5['time'] = TotalTravelled5.apply(lambda row: row['Time-Boat'][0], axis=1)
TotalTravelled5['boat'] = TotalTravelled5.apply(lambda row: row['Time-Boat'][1], axis=1)

In [277]:
del TotalTravelled5['Time-Boat']

In [278]:
TotalTravelled5

,Difference Compared To Front Runner,time,boat
0,0.0000,1630568818,4
1,0.2884,1630568818,1
2,0.4839,1630568818,5
3,2.4068,1630568818,6
4,1.7503,1630568818,3
...,...,...,...
3325,17.2575,1630608658,8
3326,22.9966,1630608658,0
3327,13.9756,1630608658,7
3328,0.0000,1630608658,9


In [279]:
TotalTravelled = TotalTravelled.merge(TotalTravelled5, on=['time','boat'])

In [280]:
TotalTravelled

,Rank 1LM,time,Distance 1st landmark,boat,Rank,Distance Travelled,latitude,longitude,heading,speed,EventProcessedUtcTime,PartitionId,EventEnqueuedUtcTime,Distance To Front Runner,Distance To Front Runner 1LM,Difference Compared To Front Runner
0,1,1630568818,2824.435159,4,1,1.427177,38.614716,-9.403606,230.835196,8.755974,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,0.000000,0.000000,0.0000
1,2,1630568818,2824.723581,1,2,1.124782,38.614134,-9.393962,243.595338,5.663308,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,0.840667,0.840667,0.2884
2,3,1630568818,2824.919089,5,3,1.058820,38.614806,-9.390425,223.677814,4.822775,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,1.145628,1.145628,0.4839
3,4,1630568818,2825.432192,9,9,0.405287,38.621923,-9.396180,234.620743,13.367485,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,1.029174,1.029174,0.9970
4,5,1630568818,2825.683277,7,8,0.658712,38.620867,-9.386242,205.598402,12.608633,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,1.656916,1.656916,1.2481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3325,6,1630608658,2741.995520,4,6,89.835278,38.053836,-10.121600,240.672691,7.652952,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,75.281285,75.281285,68.6115
3326,7,1630608658,2771.889882,6,7,59.108595,38.270832,-9.899322,224.505695,13.911123,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,106.024156,106.024156,98.5058
3327,8,1630608658,2772.983098,1,8,55.739380,38.256729,-9.827544,226.661457,9.378078,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,109.492635,109.492635,99.5990
3328,9,1630608658,2774.224431,2,9,55.303027,38.274652,-9.844028,215.303118,5.070161,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,109.785378,109.785378,100.8404


In [281]:
TotalTravelled.to_json("C:/Users/Meyke/Documents/Projecten/Vendee/0_85af9c3ad0944b8aac9dfac7666b8674_1.json", orient="records", lines=True)

In [282]:
data = pd.read_json("C:/Users/Meyke/Documents/Projecten/Vendee/0_85af9c3ad0944b8aac9dfac7666b8674_1.json", orient="records", lines=True)

In [283]:
data.head()

,Rank 1LM,time,Distance 1st landmark,boat,Rank,Distance Travelled,latitude,longitude,heading,speed,EventProcessedUtcTime,PartitionId,EventEnqueuedUtcTime,Distance To Front Runner,Distance To Front Runner 1LM,Difference Compared To Front Runner
0,1,1630568818,2824.435159,4,1,1.427177,38.614716,-9.403606,230.835196,8.755974,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,0.000000,0.000000,0.0000
1,2,1630568818,2824.723581,1,2,1.124782,38.614134,-9.393962,243.595338,5.663308,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,0.840667,0.840667,0.2884
2,3,1630568818,2824.919089,5,3,1.058820,38.614806,-9.390425,223.677814,4.822775,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,1.145628,1.145628,0.4839
3,4,1630568818,2825.432192,9,9,0.405287,38.621923,-9.396180,234.620743,13.367485,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,1.029174,1.029174,0.9970
4,5,1630568818,2825.683277,7,8,0.658712,38.620867,-9.386242,205.598402,12.608633,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,1.656916,1.656916,1.2481
